![Digital Futures](https://github.com/digital-futures-academy/DataScienceMasterResources/blob/main/Resources/datascience-notebook-header.png?raw=true)

# Setting up the Project Environment

Ultimately, the Python code written here will be extracted to scripts for execution in an automated pipeline.  To facilitate this, there is a need to set up a project environment that will allow for the execution of the code in a controlled and reproducible environment.

In the initial stages of the activities, the packages needed are `requests` and `pytest`.  The `requests` package is used to make HTTP requests to the API, while `pytest` is used for testing the code we also need *BeautifulSoup* (package name `beautifulsoup4`.  In later activities, you may need to install additional packages.  To do this, add the packages to the `pip install` command below and re-run the cell.

> **Remember:** The goal is to create a set of code cells that can be extracted to separate scripts for execution in an automated pipeline.  Therefore, the code should be kept in 3 distinct cells:
> 
> - **Shell Commands**:  Used to set up the project environment
> 
> - **Python Tests**: Used to test the Python production scripts both now and as part of the automated pipeline
> 
> - **Python Production Code**: The Python code that will be extracted to a script to execute during the pipeline

---

# Environment Setup Scripts

If you are running this notebook after cloning and have not set up your environment to run shell commands, you will need to run the following commands in your terminal to set up the environment.

> **NOTE:**  These commands need to be executed in the terminal.  
>
> Open a terminal at the root of your project before executing these commands
> 
> Until your environment is set up, Jupyter Notebooks will not be able to run **shell** scripts.

```sh
# Create a virtual environment (add the command below)
python3 -m venv .venv # Note: This command could also be python -m venv .venv # python3 and python are a symlink to the python version installed on your system

# Activate the virtual environment 
source .venv/bin/activate

# Install required package to execute shell commands from Jupyter Notebook
pip install ipykernel               ## OR 
pip install -r requirements.txt     ## IF there is already a requirements.txt file CONTAINING ipykenrnel in the project
```


In [1]:
# Install the necessary packages
!pip install requests pytest beautifulsoup4

# Create a requirements.txt file
!pip freeze > requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


> **Note:** 
> The `!` at the beginning of the lines is a special character in Jupyter Notebooks that allows you to run shell commands from the notebook.  
> These will need to be removed from any commands that are to be exported to a `.sh` shell script file for the pipeline.

---

# Python Tests

Develop any tests for functions in separate cells below.  The first has been provided for you as an example, add others as necessary.

### Test Constants

In [26]:
TEST_URL = 'DUMMY'

ERROR_STATUS = 'error'
ERROR_REQUEST_FAILED = "Request failed for URL"
ERROR_UNEXPECTED = "Unexpected error for URL"

HTML_PARSER = "html.parser"

NAV_HTML_WITH_CLASS = '<ul class="nav nav-list"><li>Item</li></ul>'

### Test `request_to_scrape`

In [3]:
# Test request_to_scrape
import pytest
from unittest.mock import patch
from requests.exceptions import Timeout, RequestException

def test_request_to_scrape_makes_correct_request():
  # Arrange
  url = 'http://books.toscrape.com/'

  with patch('requests.get') as mock_get:
        mock_get.return_value.status_code = 200
        # Act
        request_to_scrape(url)

        # Assert
        mock_get.assert_called_with(url)

def test_request_to_scrape_status_code_200():
    # Arrange
    with patch('requests.get') as mock_get:
        mock_get.return_value.status_code = 200
        mock_get.return_value.text = '<html><body><h1>Hello, World!</h1></body></html>'

        # Act
        request_to_scrape(TEST_URL)

        # Assert
        mock_get.assert_called_with(TEST_URL)


def test_request_to_scrape_status_code_404():
    # Arrange
    with patch('requests.get') as mock_get:
        mock_get.return_value.status_code = 404

        # Act
        request_to_scrape(TEST_URL)

        # Assert
        mock_get.assert_called_with(TEST_URL)

@pytest.mark.parametrize("exception, error_message",[
            (Exception('An error occured'), f"An error occured - {ERROR_UNEXPECTED}"),
            (Timeout('The request timed out'), f"The request timed out - {ERROR_REQUEST_FAILED}"),
            (RequestException('Invalid URL'), f"Invalid URL - {ERROR_REQUEST_FAILED}")
        ])

def test_request_to_scrape_exception_handling(exception, error_message):
    # Arrange
    with patch('requests.get') as mock_get:
        mock_get.side_effect = exception

        # Act
        result = request_to_scrape(TEST_URL)
        
        # Assert
        assert result == {
            "status": ERROR_STATUS,
            "error": error_message
        }

### Test `extract_element`

In [27]:
import pytest
from bs4 import BeautifulSoup

def test_HTML_with_the_specified_tag_and_class():
    # Arrange
    soup = BeautifulSoup(NAV_HTML_WITH_CLASS, HTML_PARSER)
    tag = 'ul'
    class_name = 'nav nav-list'

    # Act
    result = extract_element(soup, tag, class_name)

    # Assert
    assert result is not None
    assert result['class'] == ['nav', 'nav-list']

### Run the tests

Run the cell containing the `ipytest.run()` command to execute the tests.  The tests should all fail until you have written the production code.

Don't forget to run the installation and initialisation cell too on the first time you run the tests!


---

# Python Production Code


Develop any functions for use as production code in separate cells below. The first has been provided as an example under the Production Constants, add others as necessary.

### PRODUCTION CONSTANTS

In [5]:
# PRODUCTION CONSTANTS

# Constants for status messages
STATUS_SUCCESS = "success"
STATUS_ERROR = "error"
ERROR_NOT_HTML = "The response is not HTML"
ERROR_REQUEST_FAILED = "Request failed for URL"
ERROR_UNEXPECTED = "Unexpected error for URL"

# HTML Parser
HTML_PARSER = "html.parser"

### `request_to_scrape` Production Code

In [6]:
import requests

# request_to_scrape Production Code
def request_to_scrape(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return {
                'status': STATUS_SUCCESS,
                'data': response.text
            }
        else:
            return {
                'status': STATUS_ERROR,
                'error': ERROR_REQUEST_FAILED
            }
    except (RequestException, Timeout) as e:
        return {
            "status": STATUS_ERROR,
            "error": f"{str(e)} - {ERROR_REQUEST_FAILED}"
        }
    except Exception as e:
        return {
            "status": STATUS_ERROR,
            "error": f"{str(e)} - {ERROR_UNEXPECTED}"
        }


### `extract_book_categories` Production Code

In [7]:
# extract_book_categories Production Code

from bs4 import BeautifulSoup

def extract_book_categories(HTML_content, site):
  soup = BeautifulSoup(HTML_content, HTML_PARSER)
  nav_list = extract_element(soup, 'ul', 'nav nav-list')
  category_list = extract_element(nav_list, 'ul')

#   print(category_list.prettify())

  categories = extract_categories_and_links(category_list, site)
  for category in categories.items():
    print(category)
  return categories

### `extract_element` Production Code

In [8]:
# extract_element Production Code

from bs4 import BeautifulSoup

def extract_element(soup, tag, class_name = None):
    if class_name:
        return soup.find(tag, class_ = class_name)
    else:
        return soup.find(tag)
    
    # if soup is None:
    #     return None
    # return soup.find(tag, class_=class_name) if class_name else soup.find(tag)

### `extract_categories_and_links` Production Code

In [9]:
# extract_categories_and_links Production Code

from bs4 import BeautifulSoup

def extract_categories_and_links(category_list: BeautifulSoup, site):
    categories = {}
    for link in category_list.find_all('a'):
        category_name = link.get_text(strip=True)
        category_href = link.get('href')
        categories[category_name] = {link: f"{site}/{category_href}"}
    
    return categories

---

# Python Execution Code

Develop any code to call the developed functions below.  Add additional cells so you don't need to re-run all of the code when you develop further scripts.

In [10]:
# Python Execution Code

site = 'http://books.toscrape.com'
home_page = request_to_scrape(site)


In [11]:
extract_book_categories(home_page['data'], site)

('Travel', {<a href="catalogue/category/books/travel_2/index.html">
                            
                                Travel
                            
                        </a>: 'http://books.toscrape.com/catalogue/category/books/travel_2/index.html'})
('Mystery', {<a href="catalogue/category/books/mystery_3/index.html">
                            
                                Mystery
                            
                        </a>: 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html'})
('Historical Fiction', {<a href="catalogue/category/books/historical-fiction_4/index.html">
                            
                                Historical Fiction
                            
                        </a>: 'http://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html'})
('Sequential Art', {<a href="catalogue/category/books/sequential-art_5/index.html">
                            
                         

{'Travel': {<a href="catalogue/category/books/travel_2/index.html">
                              
                                  Travel
                              
                          </a>: 'http://books.toscrape.com/catalogue/category/books/travel_2/index.html'},
 'Mystery': {<a href="catalogue/category/books/mystery_3/index.html">
                              
                                  Mystery
                              
                          </a>: 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html'},
 'Historical Fiction': {<a href="catalogue/category/books/historical-fiction_4/index.html">
                              
                                  Historical Fiction
                              
                          </a>: 'http://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html'},
 'Sequential Art': {<a href="catalogue/category/books/sequential-art_5/index.html">
                              

---

# Jupyter Notebook Test and Linting Set Up

To run `pytest` scripts in a Jupyter Notebook cell, we need to install the `ipytest` package.  This package is NOT required for a pipeline and therefore it can be removed from the `requirements.txt` file before adding the production code to the pipeline.

To run linting, we need to install 2 packages `nbqa` and `flake8`.  We will make sure that `flake8` is included in the `requirements.txt` file when constructing the pipeline so that we can lint as part of the pipeline tests.

Run the following cell to install the `ipytest`, `nbqa` and `flake8` packages and a coverage package to help determine if all of your production code is executed during the tests!

This cell only needs to be run once (or after restarting the notebook kernel) to set up the environment for testing and linting.


In [12]:
# Install the `ipytest`, `nbqa` and `flake8` packages
!pip install ipytest nbqa flake8


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Set up `ipytest` to execute `pytest` scripts in Jupyter Notebook

In [13]:
# Configure ipytest for Jupyter Notebook

import ipytest
ipytest.autoconfig(rewrite_asserts=True, magics=True)

### Create a *config* file for `flake8`

Run this script to create a file in your project root

In [14]:
# Create a config file and ignore some flake8 rules
!echo "[flake8]" > .flake8
!echo "ignore = E402, W291, F811" >> .flake8

# Execute the tests and linting in the Jupyter Notebook

Run the following cell ***EVERY TIME*** you want to run the tests and linting that you have written in the *Python Tests* cell above.

>**Note:**
>
> This entire section does not need to be part of any pipeline scripts.  
> It is only required for the Jupyter Notebook environment during development.


## Run the tests

In [28]:
# Run the tests
ipytest.run("-vv", "-ss")

======================================= test session starts =======================================
platform win32 -- Python 3.13.0, pytest-8.3.3, pluggy-1.5.0 -- c:\Users\afzal\Desktop\Digital Futures\Python\web-scraping-sen-mo\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\afzal\Desktop\Digital Futures\Python\web-scraping-sen-mo
collecting ... collected 7 items

t_6e32693b494147aaadce7fb2213875cd.py::test_request_to_scrape_makes_correct_request PASSED
t_6e32693b494147aaadce7fb2213875cd.py::test_request_to_scrape_status_code_200 PASSED
t_6e32693b494147aaadce7fb2213875cd.py::test_request_to_scrape_status_code_404 PASSED
t_6e32693b494147aaadce7fb2213875cd.py::test_request_to_scrape_exception_handling[exception0-An error occured - Unexpected error for URL] PASSED
t_6e32693b494147aaadce7fb2213875cd.py::test_request_to_scrape_exception_handling[exception1-The request timed out - Request failed for URL] PASSED
t_6e32693b494147aaadce7fb2213875cd.py::test_request_to_scrape

<ExitCode.OK: 0>

## Run the linter

Run this script each time you want to lint your code

In [16]:
# Run the linter
!nbqa flake8 --show-source --format=pylint webscraping.ipynb

webscraping.ipynb:cell_3:2: [E402] module level import not at top of file
import pytest
^
webscraping.ipynb:cell_3:3: [E402] module level import not at top of file
from unittest.mock import patch
^
webscraping.ipynb:cell_3:4: [E402] module level import not at top of file
from requests.exceptions import Timeout, RequestException
^
webscraping.ipynb:cell_3:8: [E114] indentation is not a multiple of 4 (comment)
  # Arrange
  ^
webscraping.ipynb:cell_3:9: [E111] indentation is not a multiple of 4
  url = 'http://books.toscrape.com/'
  ^
webscraping.ipynb:cell_3:11: [E111] indentation is not a multiple of 4
  with patch('requests.get') as mock_get:
  ^
webscraping.ipynb:cell_3:12: [E117] over-indented
        mock_get.return_value.status_code = 200
        ^
webscraping.ipynb:cell_3:24: [E501] line too long (87 > 79 characters)
        mock_get.return_value.text = '<html><body><h1>Hello, World!</h1></body></html>'
                                                                             


---
